In [ ]:
%matplotlib notebook 
import json
import numpy 
import os
import sys
import time

import skimage.io
import skimage.transform
import skimage.color
import skimage.feature 
import skimage.measure

import scipy.signal
import scipy.ndimage

import matplotlib.pyplot
import matplotlib.patches

# Create bounding boxes for all non mitotic cells
This notebook performs three tasks
1. Bounding boxes for all reference cells are created 
2. All bounding boxes for mitotic cells are removed 
3. The remaining bounding boxes are enlarged by a defined offset 

In [ ]:
data_dir = '../data/'
filename_mitosis = data_dir + 'bbox_mitosis.json' 
mask_dir = data_dir + 'mask/'
image_dir = '../data/cetres.b_adapted/'
offset = 12

In [ ]:
def to_dict(x1, y1, x2, y2):
    return {
        "class": "cell",
        "x1": y1,
        "x2": y2,
        "y1": x1,
        "y2": x2
    }

def box_list(pathname):
    image = skimage.io.imread(pathname)
    region_list = skimage.measure.regionprops(image)
    return [to_dict(*region.bbox) for region in region_list]

# load image given the image number
def getImage(frameNr, image_dir):
    imageName = 'refdataB_C2_' + "%03i.png" % (frameNr,)
    image =  skimage.io.imread(image_dir + imageName)
    return skimage.color.rgb2gray(image)




# Read reference data
All reference images are loaded and all stored masks are stored as a bbox. Remember that this list includes all cells including mitotic cells!

In [ ]:
# read all cell masks 
import glob

pathnames = glob.glob(mask_dir + "ref*.png")

template_list = []

for pathname in pathnames:
    template = {
        "boxes": box_list(pathname),
        "filename": os.path.basename(pathname),
        "shape" : [1038, 1376, 3]
    }
    
    template_list += [template]

## Load bounding boxes of all mitotic cells 
These data is created using the notebook  ```2017_06_create_mitosis_bbox_template_matching```

In [ ]:
# load mitotic cells 

with open(filename_mitosis, "r") as f:
    mitosis_list = json.loads(f.read())

In [ ]:
# sort the stuff 
sorted_mitosis_list = sorted(mitosis_list, key=lambda element: element["filename"])
sorted_cell_list = sorted(template_list, key=lambda element: element["filename"])
sorted_mitosis_list[1]

## Sample image

In [ ]:
I_cell = skimage.io.imread("../data/cetres.b_adapted/refdataB_C2_014.png")

fig, ax = matplotlib.pyplot.subplots(figsize=(10, 6))
ax.imshow(I_cell,cmap="gray")     
    
minr, minc, maxr, maxc = [38, 419, 61, 442]
rect = matplotlib.patches.Rectangle((minc, minr), maxc - minc, 
                                    maxr - minr, fill = False, edgecolor = 'red', 
                                    linewidth = 2)
ax.add_patch(rect)

minr, minc, maxr, maxc = [30, 410, 71, 451]
rect = matplotlib.patches.Rectangle((minc, minr), maxc - minc, 
                                    maxr - minr, fill = False, edgecolor = 'blue', 
                                    linewidth = 2)
ax.add_patch(rect)

## Remove mitotic cells 

In [ ]:
def norm2(x1,x2,y1,y2):
    return (numpy.sqrt((x1-x2)*(x1-x2) + (y1 - y2)*(y1 - y2)))

def same_bounding_box(bbox1, bbox2):
    x1 = 1/2 * (bbox1["x1"] + bbox1["x2"]) 
    x2 = 1/2 * (bbox2["x1"] + bbox2["x2"]) 
    y1 = 1/2 * (bbox1["y1"] + bbox1["y2"]) 
    y2 = 1/2 * (bbox2["y1"] + bbox2["y2"])
    return (norm2(x1,x2,y1,y2) < 15)

def remove_duplicates(mito_list, cell_list):
    bad_cells = []

    for mito_box in mito_list["boxes"]:
        for cell_box in cell_list["boxes"]:
            if same_bounding_box(cell_box, mito_box):
                bad_cells += [cell_box]
                break

    for bad_cell in bad_cells:
        try:
            cell_list["boxes"].remove(bad_cell)
        except ValueError as e:
            continue
            
    return bad_cells

In [ ]:
duplicate_cells = []

for mito_list in sorted_mitosis_list:
    for cell_list in sorted_cell_list:
        if cell_list["filename"][-7:] == mito_list["filename"][-7:]:
            duplicate_cells += [remove_duplicates(mito_list, cell_list)] 

## Enlarge bounding boxes 

In [ ]:
x,y,z = skimage.io.imread('../data/cetres.b/refdataB_C2_001.png').shape
print(x,y)


In [ ]:

for element in sorted_cell_list:
    print(element["filename"])
    for box in element["boxes"]:
        box["x1"] = max(0, box["x1"] - offset)
        box["x2"] = min(y, box["x2"] + offset)
        box["y1"] = max(0, box["y1"] - offset)
        box["y2"] = min(x, box["y2"] + offset)
        if (box["x1"] > box["x2"]):
            print("x1 > x2!")
        if (box["y1"] > box["y2"]):
            print("y1 > y2! ")

In [ ]:
with open('../data/bbox_cells_enlarged.json', 'w') as fp:
    json.dump(sorted_cell_list, fp, indent=4)

In [ ]:
# todo: write iterator over images to find entries for 
#
def get_frame_data(cell_list, frame):
    filename = 'refdataB_C2_' + "%03i.png" % (frame)
    matches = [x for x in cell_list if x['filename'] == filename]
    if not matches:
        return []
    else:
        return(matches[0])
    
print(get_frame_data(sorted_mitosis_list,1))
print(get_frame_data(sorted_cell_list,1))

In [ ]:
# using matplotlib we can updated the figure each iteration 
fig, ax = matplotlib.pyplot.subplots(figsize=(8, 6))

frame_one = sorted_cell_list[1]

    
    
for frame in range(100,101,1):   
    # clear axis to remove patches (rect objects)
    matplotlib.pyplot.cla()
    cellImage = getImage(frame, image_dir)
    ax.imshow(cellImage,cmap='gray')
    matplotlib.pyplot.axis('off')
    # ignore frames without mitotic events
    
    frame_data = get_frame_data(sorted_cell_list, frame)
    #print(frame, frame_data)
    if frame_data:
        for box in frame_data["boxes"]:
            x1 = box['x1']
            x2 = box['x2'] 
            y1 = box['y1'] 
            y2 = box['y2']
            matplotlib.pyplot.plot([x1, x2], [y1, y1], 'r-')
            matplotlib.pyplot.plot([x1, x2], [y2, y2], 'r-')
            matplotlib.pyplot.plot([x2, x2], [y1, y2], 'r-')
            matplotlib.pyplot.plot([x1, x1], [y1, y2], 'r-')
    fig.canvas.draw()    